## Encode Images

This script uses a pre-trained deep learning model (Convolutional autoencoder) to encode raw images (png) into latent vectors. The output is saved as numpy matrix.

### set up environment

In [1]:
!pip install tensorflow-gpu

     |████████████████████████████████| 377.0MB 46kB/s 


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import cv2
from PIL import Image
import pandas as pd
import numpy as np

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image


### Set paths/ model name

In [0]:
fullPathEncoder = '/content/drive/My Drive/Model_data/Conv_AE_v4_encoder.h5'
model_name = 'Conv_AE_v4'

path_to_images = '/content/drive/My Drive/PhenoCAT/labeled_images/labeled_loose/'
path_to_save = '/content/drive/My Drive/'


### Load images

In [5]:
# get image filenames in data frame
image_list = os.listdir(path_to_images)

df_images = pd.DataFrame(data = image_list, columns = ['image_name'] )
df_images.head()

,image_name
0,CN-CP613_J06_T0001F020L01A01Z01C01_094.png
1,CN-CP613_C10_T0001F022L01A01Z01C01_077.png
2,CN-CP612_I16_T0001F019L01A01Z01C01_042.png
3,CN-CP615_I10_T0001F031L01A01Z01C01_053.png
4,CN-CP613_C07_T0001F006L01A01Z01C01_028.png


In [0]:
# save df
full_path_df = path_to_save + 'Image_names_' + model_name +'.csv'
df_images.to_csv(full_path_df)

In [6]:
dataframe = df_images
x_col = df_images.columns[0]
image_size = 256
batch_size = 12


datagen = image.ImageDataGenerator(rescale = 1./255)
                                   
pred_generator = datagen.flow_from_dataframe(dataframe = dataframe,                      
                                                directory = path_to_images,
                                                x_col = x_col,
                                                target_size = (image_size, image_size), # default: 256
                                                color_mode = 'rgb',
                                                shuffle = False, # !! keep same order as in dataframe!
                                                batch_size = batch_size,
                                                class_mode = 'input') # when images identical to input images (autoencoders)


Found 2568 validated image filenames.


### Load model

In [7]:
encoder = load_model(fullPathEncoder) 
encoder.summary()

W0728 11:04:24.074926 139942651778944 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0728 11:04:24.078903 139942651778944 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0728 11:04:24.081653 139942651778944 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtyp

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 256, 256, 64)      15552     
_________________________________________________________________
batch_normalization_17 (Batc (None, 256, 256, 64)      256       
_________________________________________________________________
activation_16 (Activation)   (None, 256, 256, 64)      0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 128, 128, 32)      51200     
_________________________________________________________________
batch_normalization_18 (Batc (None, 128, 128, 32)      128 

### Predict latent vectors, save as np array

In [9]:
latent_vectors = encoder.predict_generator(pred_generator, verbose = 1)

full_path = path_to_save + 'Latent_vectors_' + model_name

np.save(full_path, latent_vectors) 


214/214 [==============================] - 1495s 7s/step
